# Linear algebra in Julia
Based on work by Andreas Noack Jensen

## Basic linalg ops

First let's define a random matrix

In [2]:
?rand

search: rand randn transcode macroexpand @macroexpand @macroexpand1



```
rand([rng=GLOBAL_RNG], [S], [dims...])
```

Pick a random element or array of random elements from the set of values specified by `S`; `S` can be

  * an indexable collection (for example `1:9` or `('x', "y", :z)`),
  * an `AbstractDict` or `AbstractSet` object,
  * a string (considered as a collection of characters), or
  * a type: the set of values to pick from is then equivalent to `typemin(S):typemax(S)` for integers (this is not applicable to [`BigInt`](@ref)), to $[0, 1)$ for floating point numbers and to $[0, 1)+i[0, 1)$ for complex floating point numbers;

`S` defaults to [`Float64`](@ref). When only one argument is passed besides the optional `rng` and is a `Tuple`, it is interpreted as a collection of values (`S`) and not as `dims`.

!!! compat "Julia 1.1"
    Support for `S` as a tuple requires at least Julia 1.1.


# Examples

```julia-repl
julia> rand(Int, 2)
2-element Array{Int64,1}:
 1339893410598768192
 1575814717733606317

julia> using Random

julia> rand(MersenneTwister(0), Dict(1=>2, 3=>4))
1=>2

julia> rand((2, 3))
3

julia> rand(Float64, (2, 3))
2×3 Array{Float64,2}:
 0.999717  0.0143835  0.540787
 0.696556  0.783855   0.938235
```

!!! note
    The complexity of `rand(rng, s::Union{AbstractDict,AbstractSet})` is linear in the length of `s`, unless an optimized method with constant complexity is available, which is the case for `Dict`, `Set` and `BitSet`. For more than a few calls, use `rand(rng, collect(s))` instead, or either `rand(rng, Dict(s))` or `rand(rng, Set(s))` as appropriate.



In [1]:
A = rand(1:4,3,3)

3×3 Array{Int64,2}:
 4  1  3
 4  2  4
 2  3  3

Define a vector of ones

In [3]:
x = fill(1.0, (3))

3-element Array{Float64,1}:
 1.0
 1.0
 1.0

In [4]:
ones(3)

3-element Array{Float64,1}:
 1.0
 1.0
 1.0

Notice that $A$ has type Array{Int64,2} but $x$ has type Array{Float64,1}. Julia defines the aliases Vector{Type}=Array{Type,1} and Matrix{Type}=Array{Type,2}. 

Many of the basic operations are the same as in other languages
#### Multiplication

In [5]:
b = A*x

3-element Array{Float64,1}:
  8.0
 10.0
  8.0

#### Transposition
As in other languages `A'` is the conjugate transpose

In [7]:
A

3×3 Array{Int64,2}:
 4  1  3
 4  2  4
 2  3  3

In [6]:
A'

3×3 LinearAlgebra.Adjoint{Int64,Array{Int64,2}}:
 4  4  2
 1  2  3
 3  4  3

and we can get the transpose with

In [8]:
transpose(A)

3×3 LinearAlgebra.Transpose{Int64,Array{Int64,2}}:
 4  4  2
 1  2  3
 3  4  3

#### Transposed multiplication
Julia allows us to write this without *

In [9]:
A'A

3×3 Array{Int64,2}:
 36  18  34
 18  14  20
 34  20  34

#### Solving linear systems 
The problem $Ax=b$ for ***square*** $A$ is solved by the \ function.

In [11]:
A\b

3-element Array{Float64,1}:
 1.0
 1.0
 1.0

## Special Matrix Structures

Matrix structure is very important in linear algebra. To see *how* important it is, let's work with a larger linear system. Use the LinearAlgebra standard package to get access to structured matrices:

In [12]:
using LinearAlgebra

In [15]:
n = 1000
A = randn(n,n);

Julia can often infer special matrix structure

In [16]:
Asym = A + A'
issymmetric(Asym)

true

but sometimes floating point error might get in the way.

In [17]:
Asym_noisy = copy(Asym)
Asym_noisy[1,2] += 5eps()

-1.6486067070506913

In [18]:
Asym_noisy[2,1]

-1.6486067070506925

In [19]:
issymmetric(Asym_noisy)

false

Luckily we can declare structure explicitly with, for example, `Diagonal`, `Triangular`, `Symmetric`, `Hermitian`, `Tridiagonal` and `SymTridiagonal`.

In [20]:
Asym_explicit = Symmetric(Asym_noisy);

In [21]:
Asym_explicit.uplo

'U': ASCII/Unicode U+0055 (category Lu: Letter, uppercase)

In [23]:
typeof(Asym_explicit)

Symmetric{Float64,Array{Float64,2}}

In [22]:
issymmetric(Asym_explicit)

true

Let's compare how long it takes Julia to compute the eigenvalues of `Asym`, `Asym_noisy`, and `Asym_explicit`

In [24]:
@time eigvals(Asym);

  1.439223 seconds (683.14 k allocations: 42.784 MiB, 1.24% gc time)


In [25]:
@time eigvals(Asym);

  0.613704 seconds (11 allocations: 7.989 MiB)


In [26]:
@time eigvals(Asym_noisy);

  2.112630 seconds (13 allocations: 7.920 MiB)


In [28]:
@time eigvals(Asym_explicit);

  0.617841 seconds (6.82 k allocations: 8.354 MiB)


In [41]:
@time eigvals(Asym_explicit);

  0.462020 seconds (11 allocations: 7.989 MiB)


In [35]:
using BenchmarkTools

In [37]:
@btime eigvals(Asym_explicit);

  255.215 ms (11 allocations: 7.99 MiB)


In this example, using `Symmetric()` on `Asym_noisy` made our calculations about `5x` more efficient :)

#### A big problem
Using the `Tridiagonal` and `SymTridiagonal` types to store tridiagonal matrices makes it possible to work with potentially very large tridiagonal problems. The following problem would not be possible to solve on a laptop if the matrix had to be stored as a (dense) `Matrix` type.

In [43]:
n = 5;
A = SymTridiagonal(randn(n), randn(n-1))

5×5 SymTridiagonal{Float64,Array{Float64,1}}:
  2.01906   -0.158765    ⋅          ⋅          ⋅ 
 -0.158765   0.833395   2.98106     ⋅          ⋅ 
   ⋅         2.98106   -0.642389  -0.648255    ⋅ 
   ⋅          ⋅        -0.648255   1.19032   -0.62396
   ⋅          ⋅          ⋅        -0.62396    0.488835

In [44]:
A.dv

5-element Array{Float64,1}:
  2.019058227724085
  0.8333954827128094
 -0.6423886838899121
  1.1903190076243753
  0.48883454432896795

In [45]:
A.ev

4-element Array{Float64,1}:
 -0.15876453062805074
  2.9810639407166084
 -0.6482552977822438
 -0.6239598138267758

In [46]:
n = 1_000_000;
A = SymTridiagonal(randn(n), randn(n-1));
@time eigmax(A)

  2.091746 seconds (450.07 k allocations: 206.086 MiB, 2.82% gc time)


5.9841925301438526

In [48]:
n = 1_000_000;
A = SymTridiagonal(randn(n), randn(n-1));
@time eigmax(A)

  1.195237 seconds (38 allocations: 183.107 MiB, 39.98% gc time)


6.869674694219125

In [50]:
n^2, n+n-1

(1000000000000, 1999999)

In [56]:
n^2*64/8/1024/1024/1024/1024

7.275957614183426

More than 7 TB to store $n \times n$ general matrix, when $n = 1,000,00$.

In [59]:
(n+n-1)*64/8/1024/1024

15.258781433105469

Around 15 MB to store $n \times n$ symmetric tridiagonal matrix, when $n = 1,000,00$.